
# Langchain Agents -- Create your Agent in Langchain

3 tools used in this demo:
1. DuckDuckGo Search Tool
2.  Web Fetcher Tool
3. Summarizer Tool


In [ ]:
!pip install langchain openai python-dotenv requests duckduckgo-search langchainhub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: lxml
   

In [ ]:
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.1 MB/s eta 0:00:00


### 1. Importing Necessary Libraries


In [ ]:
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langchain.tools import Tool, DuckDuckGoSearchResults
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.agents import AgentType
from langchain import hub
from langchain.agents import AgentExecutor, create_xml_agent

### 2. Load API Key

In [ ]:
import os
from google.colab import userdata
from getpass import getpass

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')


### 3. Setting Up the DuckDuckGo Search Tool

In [ ]:
ddg_search = DuckDuckGoSearchResults()

### 4. Parsing HTML Content

In [ ]:
def parse_html(content) -> str:
    soup = BeautifulSoup(content, 'html.parser')
    text_content_with_links = soup.get_text()
    return text_content_with_links

### 5. Fetching Web Page Content

In [ ]:
def fetch_web_page(url: str) -> str:
  HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0'
}
  response = requests.get(url, headers=HEADERS)
  return parse_html(response.content)

### 6. Creating the Web Fetcher Tool


In [ ]:
web_fetch_tool = Tool.from_function(
    func=fetch_web_page,
    name="WebFetcher",
    description="Fetches the content of a web page"
)

### 7. Setting Up the Summarizer

In [ ]:
prompt_template = "Summarize the following content: {content}"
llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

summarize_tool = Tool.from_function(
    func=llm_chain.run,
    name="Summarizer",
    description="Summarizes a web page"
)

### 9. Initializing the Agent

In [ ]:
from langchain.agents import create_openai_tools_agent

tools = [ddg_search, web_fetch_tool, summarize_tool]
prompt = hub.pull("hwchase17/openai-tools-agent")

# Construct the XML agent
# agent = create_xml_agent(llm, tools, prompt)

# Openai Tools Agent
agent = create_openai_tools_agent(llm, tools, prompt)

### 10. Running the Agent


In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "what is Agent in LangChain? Please give a use case for the explanation"})



> Entering new AgentExecutor chain...
In LangChain, an Agent refers to an entity that interacts with the language models and performs various tasks. It acts as an interface between the user and the language models, handling the input and output of text.

A use case for the Agent in LangChain could be a language translation application. The Agent would receive text input in one language from the user and use the language models to translate it into another language. The translated text would then be returned to the user through the Agent. The Agent can handle multiple language pairs and provide accurate and efficient translations using the power of the language models.

> Finished chain.


{'input': 'what is Agent in LangChain? Please give a use case for the explanation',
 'output': 'In LangChain, an Agent refers to an entity that interacts with the language models and performs various tasks. It acts as an interface between the user and the language models, handling the input and output of text.\n\nA use case for the Agent in LangChain could be a language translation application. The Agent would receive text input in one language from the user and use the language models to translate it into another language. The translated text would then be returned to the user through the Agent. The Agent can handle multiple language pairs and provide accurate and efficient translations using the power of the language models.'}

### Test with different Agent Types

Note: The self-ask with search agent only needs one tool, see the below code:



```
ddg_search_ = [DuckDuckGoSearchResults(name="Intermediate Answer")]

# ddg_search_ = DuckDuckGoSearchResults()
prompt = hub.pull("hwchase17/self-ask-with-search")

# Construct the Self Ask With Search Agent
agent = create_self_ask_with_search_agent(llm, ddg_search_, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=ddg_search_, verbose=True, handle_parsing_errors=True)
```





In [ ]:

from langchain.chains import LLMChain
from langchain.agents import Tool
#llms
openai_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

#tools
def create_tools(llm):

    ddg_search = DuckDuckGoSearchResults()

    web_fetch_tool = Tool.from_function(
        func=fetch_web_page,
        name="WebFetcher",
        description="Fetches the content of a web page"
        )

    prompt_template = "Summarize the following content: {content}"
    llm_chain = LLMChain(
        llm=llm,
        prompt=PromptTemplate.from_template(prompt_template))

    summarize_tool = Tool.from_function(
        func=llm_chain.run,
        name="Summarizer",
        description="Summarizes a web page")

    tools = [ddg_search, web_fetch_tool, summarize_tool]

    return tools


In [ ]:
from typing import Dict, Sequence
from langchain_core.language_models import BaseLanguageModel
from langchain_core.tools import BaseTool

from langchain.agents import (create_json_agent, \
                              create_json_chat_agent,
                              create_react_agent, \
                              create_xml_agent, \
                              create_structured_chat_agent,
                              create_openai_tools_agent, \
                              create_openai_functions_agent,
                              create_xml_agent,
                              create_self_ask_with_search_agent)


import requests
from bs4 import BeautifulSoup
from langchain.tools import Tool, DuckDuckGoSearchResults
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.agents import AgentType, AgentExecutor

def create_agent(
    agent_type: str,
    llm: BaseLanguageModel,
    tools: Sequence[BaseTool],
    prompt: PromptTemplate = None
) -> AgentExecutor:
    """Create a LangChain agent executor.

    Args:
        agent_type: The type of agent to create. Options are:
            - "openai_funcs": OpenAI Functions agent
            - "openai_tools": OpenAI Tools agent
            - "xml": XML agent
            - "json_chat": JSON Chat agent
            - "react": React agent
            - "self_ask_search": Self Ask agent with search

        llm: The language model to use. Defaults to OpenAI.
        tools: The tools to equip the agent with.
        prompt: A prompt template to initialize the agent with.
            If none provided, will pull a template from Hub.

    Returns:
        An executor for the created agent.
    """
    # Map agent types to creation functions
    CREATE_AGENT_FUNCS = {"openai_funcs": create_openai_functions_agent,
                       "openai_tools": create_openai_tools_agent,
                       "xml": create_xml_agent,
                       "json_chat":create_json_chat_agent ,
                       "structured": create_structured_chat_agent,
                       "react": create_react_agent,
                       "self_ask_search": create_self_ask_with_search_agent
                      }

    # Get the agent creation function
    create_agent_func = CREATE_AGENT_FUNCS[agent_type]

    if not prompt:
        if agent_type=='openai_funcs':
            prompt = hub.pull("hwchase17/openai-functions-agent")
        elif agent_type=='openai_tools':
            prompt = hub.pull("hwchase17/openai-tools-agent")
        elif agent_type=='xml':
            prompt = hub.pull("hwchase17/xml-agent-convo")
        elif agent_type=='json_chat':
            prompt = hub.pull("hwchase17/react-chat-json")
        elif agent_type=='structured':
            prompt = hub.pull("hwchase17/structured-chat-agent")
        elif agent_type=='react':
            prompt = hub.pull("hwchase17/react")
        elif agent_type=='self_ask_search':
            prompt = hub.pull("hwchase17/self-ask-with-search")
    # try:
    #     print(prompt.get_prompts()[0].template)
    # except:
    #     print(prompt.messages)
    # finally:
    #     pass

    # Return an agent executor by passing in the agent and tools
    if agent_type=='self_ask_search':
      ddg_search_ = [DuckDuckGoSearchResults(name="Intermediate Answer")]

      prompt = hub.pull("hwchase17/self-ask-with-search")

      # Construct the Self Ask With Search Agent
      agent = create_self_ask_with_search_agent(llm, ddg_search_, prompt)
      # Create an agent executor by passing in the agent and tools
      return AgentExecutor(agent=agent, tools=ddg_search_, verbose=True, handle_parsing_errors=True)

    else:
      agent = create_agent_func(llm, tools, prompt)
      return AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)


In [ ]:
def parse_html(content) -> str:
    soup = BeautifulSoup(content, 'html.parser')
    text_content_with_links = soup.get_text()
    return text_content_with_links

def fetch_web_page(url: str) -> str:
    HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0'}
    response = requests.get(url, headers=HEADERS)
    return parse_html(response.content)

In [ ]:
agent_type_dict = {"openai_funcs": create_openai_functions_agent,
                   "openai_tools": create_openai_tools_agent,
                   "xml": create_xml_agent,
                   "json_chat":create_json_chat_agent ,
                   "structured": create_structured_chat_agent,
                   "react": create_react_agent,
                   "self_ask_search": create_self_ask_with_search_agent
                  }

In [ ]:
llm = openai_llm

tools = create_tools(llm)
# agent = create_agent('xml', llm, tools)


In [ ]:
query = "What is the value of cash and cash equivalents for Tesla in 2022?"

for agent_type_key in agent_type_dict.keys():
  agent = create_agent(agent_type_key, llm, tools)
  results = agent.invoke({"input": query})
  print(f'Agent Type: {agent_type_key}\n\nQuestion: {query}\n\nAnswer👇👇👇: \n{results}')



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'Tesla cash and cash equivalents 2022'}`


[snippet: 0.56. 0.49. Upgrade. Source: Financials are provided by Nasdaq Data Link and sourced from the audited annual ( 10-K) and quarterly ( 10-Q) reports submitted to the Securities and Exchange Commission (SEC). Detailed balance sheet for Tesla, Inc. (TSLA), including cash, debt, assets, liabilities, and book value., title: Tesla, Inc. (TSLA) Balance Sheet - Stock Analysis, link: https://stockanalysis.com/stocks/tsla/financials/balance-sheet/], [snippet: 2022: Cash Flows from Operating Activities: ... Effect of exchange rate changes on cash and cash equivalents and restricted cash (142) (567) Net (decrease) increase in cash and cash equivalents and restricted cash ... ("Tesla", the "Company", "we", "us" or "our"), including the consolidated balance sheet as of September 30 ..., title: tsla-20230930 - SEC.gov, link: https://www.sec.gov/Archives/edga

MissingSchema: Invalid URL "Tesla's financial statements for 2022": No scheme supplied. Perhaps you meant https://Tesla's financial statements for 2022?